# 第三章：複合型態（Composite Types）  
> 本章完整說明 **陣列 (array)**、**切片 (slice)**、**字串 (string)／rune／byte**、**map**、**struct** 的語意、最佳實務、常見陷阱與多種使用情境。  
> 說明以 **台灣正體中文** 撰寫，專業名詞採「中文（英文）」雙語。

## 陣列—很難直接使用（Arrays）
**陣列 (array)** 是「固定長度（fixed-length）且同質（homogeneous）」的連續記憶體集合；其**長度是型別的一部分**。  
實務上較少直接使用，原因：

- **長度固定**：宣告後無法擴充或縮短。
- **值語意（value semantics）**：**傳遞或指定給變數時會整份複製**，成本可能高。
- **型別嚴格**：`[3]int` 與 `[4]int` 是不同型別，無法互相指派。
- **不支援 `append`**：想動態增長需改用 **slice**。
- 常用於：**固定大小**資料、**FFI** 或 **需要在堆疊配置的高效連續記憶體**。

> 小提醒：**多維陣列**是陣列的陣列（array of arrays），尺寸都固定。

## 宣告
```go

//會預設給一個空的array, 都塞成0
var x1 [3]int 

//可以直接assign
var x2 = [3]int{10,20,30}



```

In [3]:
//go:build ignore
package main

import "fmt"

func mutateArray(a [3]int) { // 值傳遞（copy）
	// 這裡修改不會影響呼叫端
	a[0] = 99
}

func mutateArrayPtr(a *[3]int) { // 指標傳遞（reference-like）
	a[0] = 77
}

func main() {
	// 1) 基本宣告與推導大小
	var a1 [3]int = [3]int{1, 2, 3}
	a2 := [...]int{10, 20, 30} // 由編譯器推導長度為 3
	fmt.Println("a1:", a1, "a2:", a2)

	// 2) 值語意：賦值會複製整個陣列
	a3 := a1
	a3[0] = 999
	fmt.Println("a1 不受影響:", a1, "a3:", a3)

	// 3) 呼叫函式：值傳遞 vs 指標傳遞
	mutateArray(a1)
	fmt.Println("after mutateArray(a1):", a1) // 仍為原值

	mutateArrayPtr(&a1)
	fmt.Println("after mutateArrayPtr(&a1):", a1) // 被修改

	// 4) 多維陣列（固定大小）
	var grid [2][3]int
	grid[0][1] = 42
	fmt.Println("grid:", grid)
}

a1: [1 2 3] a2: [10 20 30]
a1 不受影響: [1 2 3] a3: [999 2 3]
after mutateArray(a1): [1 2 3]
after mutateArrayPtr(&a1): [77 2 3]
grid: [[0 42 0] [0 0 0]]


## Slice（切片）
**Slice** 是建立在「底層陣列（backing array）」之上的**視圖（view）**，包含三個欄位：`ptr`、`len`、`cap`。  
- **長度（len）**：可讀可改（透過切割），表示可見元素數。
- **容量（cap）**：從 `ptr` 開始到底層陣列可用的最大範圍。
- **動態增長**：透過 `append` 可能導致 **重新配置（reallocate）** 新底層陣列。  
- **零值（zero value）**：`nil` slice（`var s []T`），`len=0`、`cap=0`、`s==nil` 為真。  
- **空 slice**：`[]T{}` 或 `make([]T, 0)`，`len=0`、`cap>=0`、`s!=nil`。

> 直覺心法：**用 slice，除非你真的需要固定長度 array 的語意。**

### len（長度）
- `len(s)` 回傳目前可見元素數量。  
- 切片運算 `s[i:j]` 會回傳 **新的 slice header**（共享同一個底層陣列）。

In [18]:
//go:build ignore
package main

import "fmt"

func main() {
	s := []int{1, 2, 3, 4}
	fmt.Println("len:", len(s)) // 4

	s2 := s[:2]       // 共享底層陣列
	fmt.Println(s2)   // [1 2]
	fmt.Println(len(s2), cap(s2)) // 2, cap>=2（視原切片容量而定）
}

len: 4
[1 2]
2 4


### 宣告 slice（宣告與零值）
- `var s []T`：零值為 `nil` slice，可安全 `append`。  
- 字面值：`s := []T{...}` 立即帶值。
- 還有使用 []s(nil) 來init value

In [13]:
//go:build ignore
package main

import "fmt"

func main() {
	var s []int // nil slice
	fmt.Println(s == nil, len(s), cap(s)) // true 0 0
	s = append(s, 1, 2, 3)
	fmt.Println(s)

	s2 := []string{"go", "rust"}
	fmt.Println(s2)
}

true 0 0
[1 2 3]
[go rust]


### make（建立 slice）
- `make([]T, len)`：建立長度=容量的 slice。  
- `make([]T, len, cap)`：建立指定長度與容量的 slice。

In [12]:
//go:build ignore
package main

import "fmt"

func main() {
	a := make([]int, 3)       // [0 0 0] len=3 cap=3
	b := make([]int, 0, 4)    // [] len=0 cap=4，適合後續 append
	fmt.Println(a, len(a), cap(a))
	fmt.Println(b, len(b), cap(b))
}

[0 0 0] 3 3
[] 0 4


### append（動態擴充）
- `append(s, v...)` 會回傳**新 slice**（可能仍共享，也可能已搬家）。  
- 若容量不足，**執行期會配置新陣列**並複製舊資料，原 `s` 與新切片不再共享。  
- **常見陷阱**：對「子切片」`s2 := s[:k]` 做 `append`，可能覆蓋 `s` 之後元素（尚在同一底層陣列且容量足夠）。

In [ ]:
//go:build ignore
package main

import "fmt"

func main() {
	base := []int{1, 2, 3, 4}
	sub := base[:2] // [1 2]，cap 可能大於 2
	sub = append(sub, 99) // 若 cap 夠，會寫回同一底層陣列
	fmt.Println("base 可能被覆寫:", base)
	fmt.Println("sub:", sub)

	// 若想避免彼此影響，預先 copy 出獨立底層陣列
	isolated := append([]int(nil), base[:2]...)
	isolated = append(isolated, 88)
	fmt.Println("base 不受影響:", base)
	fmt.Println("isolated:", isolated)
}

base 可能被覆寫: [1 2 99 4]
sub: [1 2 99]
base 不受影響: [1 2 99 4]
isolated: [1 2 88]


### 容量（cap, capacity）
- `cap(s)` 代表自 `ptr` 起到底層陣列可用的總長度。  
- **成長策略**由實作決定（通常逐步倍增），不要依賴精確倍數。  
- 可用三索引切片語法 `s[i:j:k]` 控制新切片容量為 `k-i`（**full slice expression**）。

In [11]:
//go:build ignore
package main

import "fmt"

func main() {
	s := make([]int, 2, 5) // len=2, cap=5
	fmt.Println(s, len(s), cap(s))

	// full slice：限制容量，避免外溢覆寫
	s2 := s[:2:2]        // len=2, cap=2
	s2 = append(s2, 7)   // 會強制配置新底層陣列
	fmt.Println("s:", s, "s2:", s2, "cap(s2):", cap(s2))
}

[0 0] 2 5
s: [0 0] s2: [0 0 7] cap(s2): 4


### 切割 slice（slicing）
- 基本語法：`s[i:j]`，含 `i` 不含 `j`。  
- 省略 `i` 視為 `0`，省略 `j` 視為 `len(s)`。  
- 使用 `s[i:j:k]` 可以控制容量，避免 `append` 回寫影響原切片。

In [14]:
//go:build ignore
package main

import "fmt"

func main() {
	s := []int{1,2,3,4,5}
	fmt.Println(s[1:3])   // [2 3]
	fmt.Println(s[:2])    // [1 2]
	fmt.Println(s[2:])    // [3 4 5]
	fmt.Println(s[0:0:0]) // len=0 cap=0 的切片
}

[2 3]
[1 2]
[3 4 5]
[]


### 將陣列轉換成 slice（array → slice）
對陣列或指向陣列的指標做切片即可獲得 slice。

In [16]:
//go:build ignore
package main

import "fmt"

func main() {
	arr := [5]int{1,2,3,4,5}
	s1 := arr[1:4]      // 基於陣列的切片
	s2 := (&arr)[2:5]   // 以 *[5]int 也可切
	fmt.Println(s1, s2)
}

[2 3 4] [3 4 5]


### copy（複製 slice）
- `n := copy(dst, src)`：回傳實際複製數量（`min(len(dst), len(src))`）。  
- **用於**：擴容前資料搬移、切片去共享、環形緩衝等。

In [17]:
//go:build ignore
package main

import "fmt"

func main() {
	src := []int{1,2,3,4}
	dst := make([]int, 2)
	n := copy(dst, src) // 只會複製 2 個
	fmt.Println("n:", n, "dst:", dst)

	// 去共享：
	b := []int{10,20,30,40}
	isolated := make([]int, len(b))
	copy(isolated, b) // isolated 與 b 不共享底層陣列
	b[0] = 999
	fmt.Println("b:", b, "isolated:", isolated)
}

n: 2 dst: [1 2]
b: [999 20 30 40] isolated: [10 20 30 40]


## 字串與 rune 與 byte（Strings, Runes, Bytes）
- **字串（string）**：**不可變（immutable）**的 **UTF-8** 位元組序列。索引操作回傳 **byte**。  
- **byte**：`uint8` 的別名，表示單一位元組。  
- **rune**：`int32` 的別名，表示一個 **Unicode code point**。  

重點：
- `len(str)` 回傳的是 **位元組數（bytes）**，不是字元數。  
- 以 `for range` 迭代字串會以 **rune** 為單位前進。  
- 需要以「字元」為單位處理時，先轉 `[]rune`。  
- 可用 `utf8.RuneCountInString` 計算字元（rune）數。

In [ ]:
//go:build ignore
package main

import (
	"fmt"
	"unicode/utf8"
)

func main() {
	str := "Hello, 世界🌏"
	fmt.Println("bytes(len):", len(str))               // 位元組數
	fmt.Println("runes:", utf8.RuneCountInString(str)) // 字元數

	// 逐 rune 迭代
	for i, r := range str {
		fmt.Printf("i=%d r=%c U+%04X\n", i, r, r)
	}

	// 以 rune 修改字串（需轉換，因 string 不可變）
	runes := []rune(str)
	runes[7] = '界'
	fmt.Println("修改後：", string(runes))

	// 高效拼接：strings.Builder / bytes.Buffer
}

## map（映射 / 連想陣列）
**map[K]V** 是雜湊表（hash table）實作的鍵值對集合。

重點：
- **零值為 `nil`**，`var m map[K]V` 可讀不可寫；需 `make(map[K]V)` 後才能賦值。
- **讀/寫平均 O(1)**；**迭代順序不保證固定**（Go 1.12 起更刻意隨機化）。
- key 限制：**必須可比較（comparable）**；因此 `slice/map/function` 不能做 key。
- **非併發安全**：多 goroutine 同時寫需加鎖或使用 `sync.Map`（特殊情境適用）。

### 讀取和寫入 map

In [ ]:
//go:build ignore
package main

import "fmt"

func main() {
	m := make(map[string]int)
	m["alice"] = 25
	m["bob"] = 30

	fmt.Println("alice =", m["alice"]) // 25

	// 讀取不存在的 key 回傳 value 的零值
	fmt.Println("unknown =", m["unknown"]) // 0
}

### 逗號 ok 寫法（comma-ok idiom）
用於**判斷 key 是否存在**，避免把零值誤判為有值。

In [ ]:
//go:build ignore
package main

import "fmt"

func main() {
	m := map[string]int{"x": 1}
	v, ok := m["y"]
	if !ok {
		fmt.Println("y 不存在")
	} else {
		fmt.Println("y =", v)
	}
}

### 刪除 map 內容（delete）
- `delete(m, key)`：不存在也不會出錯。

In [ ]:
//go:build ignore
package main

import "fmt"

func main() {
	m := map[string]int{"a":1, "b":2}
	delete(m, "a")
	delete(m, "zzz") // 安全
	fmt.Println(m)   // 只剩 b
}

### 將 map 當成 set 來使用（Set via map）
- 推薦：`map[T]struct{}` 節省記憶體（空 struct 不佔位）。  
- 若需表示「是否啟用」可用 `map[T]bool`。

In [ ]:
//go:build ignore
package main

import "fmt"

func main() {
	// 1) 使用 struct{} 作為集合
	set := make(map[string]struct{})
	set["go"] = struct{}{}
	set["rust"] = struct{}{}

	_, hasGo := set["go"]
	fmt.Println("has go:", hasGo)

	// 2) 使用 bool 作為集合
	set2 := map[int]bool{1:true, 3:true}
	fmt.Println("1 in set2:", set2[1])
}

## struct（結構體）
**struct** 用於定義**聚合資料（aggregate data）**。

重點：
- 可含 **欄位標籤（tag）** 提供序列化等額外資訊（例如 `json:"name,omitempty"`）。
- **方法（method）** 可以綁定在型別上，常以指標接收者實作（避免拷貝、允許修改）。
- **比較（==）**：當 **所有欄位皆可比較** 時，struct 可直接比較。
- **型別相等性（type identity）**：不同命名型別即使欄位相同也**不是同型別**，需**顯式轉型**。

In [ ]:
//go:build ignore
package main

import (
	"encoding/json"
	"fmt"
)

type Person struct {
	Name string `json:"name"`
	Age  int    `json:"age,omitempty"`
}

func (p *Person) Birthday() { // 指標接收者：修改原物件
	p.Age++
}

func main() {
	p := Person{Name:"Alice", Age:20}
	fmt.Println(p)

	p.Birthday()
	fmt.Println("after:", p)

	// JSON：欄位須導出（大寫開頭）才會被序列化
	b, _ := json.Marshal(p)
	fmt.Println(string(b))
}

### 匿名 struct（Anonymous Struct）
適合**臨時資料**或**測試**場景，不必事先命名型別。

In [ ]:
//go:build ignore
package main

import "fmt"

func main() {
	user := struct {
		ID   int
		Name string
	}{ID: 1001, Name: "Bob"}

	fmt.Printf("%#v\n", user)
}

### 比較與轉換 struct（Equality & Conversion）
- **比較**：所有欄位皆可比較時，`==` 與 `!=` 合法。含 slice/map/function 欄位時不可比較。  
- **轉換**：不同命名型別需顯式轉換；且欄位集合、順序、型別必須相同（或底層型別相同）。

In [ ]:
//go:build ignore
package main

import "fmt"

type Point struct{ X, Y int }
type MyPoint struct{ X, Y int }

func main() {
	p1 := Point{1,2}
	p2 := Point{1,2}
	fmt.Println("p1==p2 ?", p1 == p2) // true（欄位皆可比較）

	var mp MyPoint = MyPoint(p1) // 顯式轉換
	fmt.Println(mp)
}

## 總結
- **陣列（array）**：固定長度、值語意；常被 **slice** 取代。  
- **slice**：日常主力。理解 `len/cap/backing array/append/copy/full slice` 才能避免共享陷阱。  
- **字串／rune／byte**：字串不可變、UTF-8；以 **rune** 處理字元層級邏輯。  
- **map**：鍵值結構、讀寫 O(1)、隨機迭代、非併發安全；以 `comma-ok` 判斷存在性。  
- **struct**：自訂資料模型，支援 method、tag、比較與顯式轉換。

> 下一步建議：練習撰寫以 `slice + map + struct` 串接的小型資料處理程式，並特別觀察 **子切片 append 的共享影響** 與 **map 的 comma-ok**。